In [2]:
import sys
import numpy as np
import pandas as pd
import folium

In [4]:
data_path = "./osm/amenities-vancouver.json.gz"
data = pd.read_json(data_path, lines = True)

In [5]:
food = data[data['amenity'].str.contains("restaurant|food|cafe|pub|bar|ice_cream|food_court|bbq|bistro") & ~data['amenity'].str.contains("disused")]
food = food.dropna()
food = food[food.apply(lambda x: 'opening_hours' in x['tags'], axis = 1)]
food = food[['lat', 'lon', 'amenity', 'tags']]